# Create a lookup table with IRS stats by Zipcode

From https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi

Fix the file: irs2016byZipCode.csv
to be in the zip code format for the lending club data (94107 to 921xx)

We will summarize and average the zip codes



In [10]:
from __future__ import print_function 
from __future__ import division

# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#import plotly.offline as py
#py.init_notebook_mode(connected=True)
#import plotly.graph_objs as go
#import plotly.tools as tls
import itertools
import datetime

import warnings
warnings.filterwarnings('ignore')

SEED = 42

# Load the data from the IRS

In [3]:
df = pd.read_csv('https://www.irs.gov/pub/irs-soi/16zpallagi.csv')
#df = pd.read_csv('irs2016byZipCode.csv')

df.sample(3)

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
1129,1,AL,35611,2,2710,1110,840,680,1540,5780,...,2130,5114,0,0,0,0,380,473,2300,6480
149161,47,TN,38567,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18692,6,CA,95912,2,730,270,250,190,470,2090,...,540,1280,0,0,0,0,120,184,600,1481


# Fix The Zipcode data

The data in the lending club file is in the format 941xx for a the zipcode: 94107 

In [4]:
def fixZip(val):
    zip3 = int(val / 100)
    x = '{0:03d}xx'.format(zip3)
    return x


In [5]:
df['zipcode3'] = df['zipcode'].apply(fixZip)
df.sample(5)

,STATEFIPS,STATE,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,...,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902,zipcode3
15836,6,CA,94573,2,90,70,30,40,50,190,...,239,0,0,0,0,60,83,60,146,945xx
57898,21,KY,40363,4,30,0,20,0,0,60,...,305,0,0,0,0,0,0,0,0,403xx
111609,36,NY,14580,2,5370,3320,1250,670,2620,8410,...,13421,0,0,0,0,830,1319,4440,9661,145xx
135648,42,PA,16852,1,70,40,0,0,40,110,...,16,0,0,0,0,0,0,50,69,168xx
158454,48,TX,79052,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,790xx


# Sum the data for the new 'sub-total' zipcodes

In [6]:
x = df.groupby(by='zipcode3').sum()
display(x)

,STATEFIPS,zipcode,agi_stub,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
zipcode3,,,,,,,,,,,,,,,,,,,,,
000xx,8862,0,1071,147025790,69557310,53211080,21300200,78415000,284407960,93313960,...,112633470,1497791185,3539690,7420047,3724310,16281907,28195140,148115633,111888070,341632930
010xx,8975,377273,1256,221930,115150,79750,23580,117920,390630,105760,...,176370,1803431,3260,4285,3710,7997,39150,161069,172540,449228
011xx,1800,80322,252,72360,36040,16720,18470,34970,135690,50880,...,48370,452516,930,1826,970,3819,9030,37478,61210,188755
012xx,4500,224010,630,62810,33200,21590,6860,32820,106200,26520,...,48620,504458,750,1158,1160,4985,10940,49169,47840,120062
013xx,3900,211044,546,39810,20640,14440,3970,20170,69170,17370,...,31430,244641,140,169,260,669,6910,20843,30870,74881
014xx,3300,191652,462,105130,51610,39390,12140,52490,193200,58010,...,83830,1005370,2540,3222,2350,7142,18350,85019,82730,227520
015xx,6750,415674,945,193370,92730,79970,17690,104380,362220,107540,...,159440,2380447,7680,10453,6780,11405,37840,211458,147450,418762
016xx,1650,106062,231,87130,47320,22120,16000,47300,152810,49510,...,64570,498248,770,918,800,2152,12310,43338,72260,204713
017xx,4050,282738,567,194140,90750,86310,13500,104490,376360,115770,...,162360,5033937,18620,39782,17590,75637,51080,453176,128950,518848


# We only want to keep a a select few columns

  - PREP	Number of returns with paid preparer's signature
  - N2	Number of exemptions
  - ELDERLY	Number of elderly returns
  - A00100	Adjust gross income (AGI)
  - N02650	Number of returns with total income
  - A02650	Total income amount
  - N00200	Number of returns with salaries and wages
  - A00200	Salaries and wages amount
  - N00300	Number of returns with taxable interest
  - A00300	Taxable interest amount
  - N00600	Number of returns with ordinary dividends 
  - A00600	Ordinary dividends amount
  - N00650	Number of returns with qualified dividends
  - A00650	Qualified dividends amount
  - N00900	Number of returns with business or professional net income (less loss)
  - A00900	Business or professional net income (less loss) amount
  - N01000	Number of returns with net capital gain (less loss)
  - A01000	Net capital gain (less loss) amount
  - SCHF  	Number of farm returns
  - N02300	Number of returns with unemployment compensation
  - A02300	Unemployment compensation amount

In [7]:
final =pd.DataFrame(x, columns=['PREP','N2','ELDERLY','A00100','N02650','A02650','N00200','A00200','N00300',
          'A00300','N00600','A00600','N00650','A00650','N00900','A00900','N01000','A01000','SCHF',
          'N02300','A02300'])


In [8]:
display(final)

,PREP,N2,ELDERLY,A00100,N02650,A02650,N00200,A00200,N00300,A00300,...,A00600,N00650,A00650,N00900,A00900,N01000,A01000,SCHF,N02300,A02300
zipcode3,,,,,,,,,,,,,,,,,,,,,
000xx,78415000,284407960,34562930,10250219281,147025470,10396138255,122834750,7101762895,41417350,77925176,...,222918946,24807200,174406588,24351540,336239587,23142210,491383020,1584600,5599950,25751489
010xx,117920,390630,62540,14043321,221930,14253798,184770,10039847,81570,88909,...,223812,43510,175773,30330,514207,35590,336391,500,12640,77820
011xx,34970,135690,13980,3639749,72360,3681669,63770,2690506,14320,22030,...,62171,7550,51472,7430,105856,6180,188322,0,4620,25045
012xx,32820,106200,20160,3838583,62810,3904876,50690,2458102,21200,28558,...,117035,14840,94355,9370,159169,11770,170941,100,3390,19107
013xx,20170,69170,12340,2159023,39810,2194060,32750,1502007,14240,11528,...,42697,7650,32965,6300,90067,6440,45151,310,2190,13521
014xx,52490,193200,24910,7333511,105130,7427453,91090,5623021,41170,32521,...,99061,18380,75421,13970,245946,16300,224568,60,6230,41842
015xx,104380,362220,47490,15902817,193370,16111396,166880,12300492,80820,70075,...,200214,42140,154388,26730,513767,37400,422501,160,10770,73400
016xx,47300,152810,16210,4385015,87130,4434072,77060,3506213,20850,16841,...,50940,9350,39725,10810,140392,8260,69953,0,4630,28301
017xx,104490,376360,50860,25980191,194140,26399833,160750,17083920,90480,231507,...,882641,65340,695213,37430,1042929,61180,2022716,60,8280,63488


In [9]:
final.to_csv('2016IRSZipCode3X.csv')